In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Import packages
import time, copy
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, models, transforms
from torchvision.utils import make_grid
from torchvision.models.vgg import model_urls
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler

from os.path import isfile, join, abspath, exists, isdir, expanduser

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv', 'train_images', 'test_images']


In [2]:
# Class names
class_names = ['empty', 'deer', 'moose', 'squirrel', 'rodent', 'small_mammal', 'elk', 'pronghorn_antelope', 
               'rabbit', 'bighorn_sheep', 'fox', 'coyote', 'black_bear', 'raccoon', 'skunk', 'wolf', 
               'bobcat', 'cat', 'dog', 'opossum', 'bison', 'mountain_goat', 'mountain_lion']

# csv file & data file path

test_csv_file = '../input/test.csv'
test_data_dir = '../input/test_images'
test_df = pd.read_csv(test_csv_file)

In [3]:
test_df.to_pickle('test_df.pkl')

In [4]:
class IWildCamDataset(Dataset):
    
    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            df (pandas.DataFrame): Contains basic information.
            root_dir (string): The path where image data is saved.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir,
                                self.df.iloc[idx].file_name)
        with open(img_path, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
        
        if('category_id' in self.df.iloc[idx]):
            category = self.df.iloc[idx].category_id
        else:
            # In test set, there is no given category. Here we will not return the category,
            # return the img_id instead. (Because we need to keep track of the img id during
            # testing)
            category = self.df.iloc[idx].id
        
        if('file_name' in self.df.iloc[idx]):
            filename = self.df.iloc[idx].file_name

        # Transform
        if self.transform:
            image = self.transform(image)

        return image, category, filename

In [5]:
# Data transforms
data_transforms = transforms.Compose([
    transforms.Resize(128),  # 1. Resize smallest side to 128.
     transforms.CenterCrop(128), # 2. Crop the center 128x128 pixels.
     transforms.ToTensor(), # 3. Convert to pytorch tensor.
     transforms.Normalize(mean = [0.485, 0.456, 0.406],  # normalize.
                          std = [0.229, 0.224, 0.225])
])

# Get dataset
test_set  = IWildCamDataset(test_df, test_data_dir, transform=data_transforms)

In [6]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install opencv-contrib-python
import cv2
print(os.listdir("../input"))
import matplotlib.pyplot as plt

import scipy.misc
import zipfile

%matplotlib inline 

    100% |████████████████████████████████| 32.6MB 1.0MB/s 
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
['train.csv', 'sample_submission.csv', 'test.csv', 'train_images', 'test_images']


In [7]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
wb = cv2.xphoto.createSimpleWB()
wb.setP(0.4)

In [8]:
startdir = "p_test_set"  #要压缩的文件夹路径
file_news = startdir +'.zip' # 压缩后文件夹的名字
azip = zipfile.ZipFile(file_news, 'w')
root_dir = '../input/test_images/'
desired_size = 256

for idx in range(0, len(test_set)):
    test_img = root_dir + test_set[idx][2]
    temp_img = cv2.imread(test_img, cv2.IMREAD_COLOR)        
    
    img_wb = wb.balanceWhite(temp_img)

    img_lab = cv2.cvtColor(img_wb, cv2.COLOR_BGR2Lab)

    l, a, b = cv2.split(img_lab)
    res_l = clahe.apply(l)
    res = cv2.merge((res_l, a, b))

    res = cv2.cvtColor(res, cv2.COLOR_Lab2BGR)
    resized = cv2.resize(res, (desired_size,)*2).astype('uint8')
    
    imgname = test_set[idx][2]
    scipy.misc.imsave(imgname, resized)
    azip.write(imgname, compress_type=zipfile.ZIP_LZMA)
    
    os.remove(imgname)

azip.close()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
